# Sample LISA (Lidar Light Scattering Augmentation) Usage For Noisy Data Generation

## 📚 Import Libraries 📚

In [1]:
from extern.atmos_models import *
import open3d as o3

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Define Lisa Model

In [8]:
lisa = LISA(atm_model='rain',rmax=100)

## 💭 Noisy Point Cloud Generation 💭

### 🛠️ Utils 🛠️

In [9]:
def visualize_cloud(clean_lidar_points):
    # Convert the numpy array to a Vector3dVector
    point_cloud = o3.utility.Vector3dVector(clean_lidar_points)
    # You can now use point_cloud with other Open3D functions and methods
    pcd = o3.geometry.PointCloud()
    pcd.points = point_cloud
    o3.visualization.draw_geometries([pcd])

def analysis_noisy_data(clean_data_,noisy_data_):
    
    lost_points_indices = np.where(noisy_data[:, 4] == 0)[0]
    unlost_points_indices = np.where(noisy_data[:, 4] != 0)[0]
    
    noisy_point_cloud_ = np.delete(clean_lidar_points, lost_points_indices, axis=0)
    removed_points_ = np.delete(clean_lidar_points, unlost_points_indices, axis=0)

    print(f"{removed_points_.shape[0]} points have been removed from clean data due to rain.So the clean LIDAR point cloud decreased to {noisy_point_cloud_.shape[0]} points due to rain.")
    return noisy_point_cloud_,removed_points_

### 🚀 Read and visualize clean data from dataset 🚀

In [10]:
clean_lidar_data = np.load("sample_clean_data.npz")

In [11]:
clean_lidar_points= clean_lidar_data['pcloud_points']
num_points, num_dimensions = clean_lidar_points.shape
print(f"The clean LIDAR point cloud has {num_points} points, each with {num_dimensions} dimensions.")
visualize_cloud(clean_lidar_points)

The clean LIDAR point cloud has 18120 points, each with 3 dimensions.


### ☔ Obtain noisy point cloud with Lisa ☔

In [12]:
clean_lidar_reflectance= clean_lidar_data['pcloud_attr.reflectance']
#Normalize lidar reflectance to Lisa usage
normalized_lidar_reflectance= (clean_lidar_reflectance - clean_lidar_reflectance.min()) / (clean_lidar_reflectance.max() - clean_lidar_reflectance.min())

In [13]:
clean_data = np.column_stack((clean_lidar_points, normalized_lidar_reflectance))
noisy_data = lisa.augment(clean_data,60) # rain rate 33 mm/hr

In [14]:
noisy_point_cloud,removed_points=analysis_noisy_data(clean_data,noisy_data)

4004 points have been removed from clean data due to rain.So the clean LIDAR point cloud decreased to 14116 points due to rain.


### 〰️ Visualize noisy point cloud 〰️

In [15]:
visualize_cloud(noisy_point_cloud)

### 💧 Visualize removed points due to rain drop 💧

In [16]:
visualize_cloud(removed_points)